# Work on #2017Elyséefr data

imports :

In [1]:
import pandas as pd

import datetime

#Twitter intelligence tool to scrap data from twitter : 
#https://github.com/twintproject/twint
import twint

#Pour gérer une erreur : 
#Voir https://github.com/twintproject/twint/issues/166
import nest_asyncio
nest_asyncio.apply()

# Reading and describing data :

Les fichiers sont en .tab donc il faut ajuster les paramètres de `pd.read_csv` :

In [2]:
dfprofiles=pd.read_csv('Donnees/data_elysee2017_tweets/profiles_annotations.tab', sep='\t')

In [3]:
dfprofiles.head()

,FROM_USER_ID,PROFILE_NATURE,PARTY,MEDIA_PROFESSIONAL,SEX
0,1264925822,individual,fi,NaN,f
1,1265204928,individual,fi,NaN,f
2,1265310602,individual,fi,NaN,m
3,1265661672,individual,fi,NaN,m
4,1267543572,individual,fi,NaN,m


In [4]:
dfprofiles.shape

(22853, 5)

#### Données sur le parti politique : 

In [5]:
dfprofiles['PARTY'].isna().sum()

3401

Il n'y a que 3401 nans pour le parti politique sur les 22853 profils disponibles. 

Etant donné qu'il serait très compliqué de les estimer (sauf à la main) donc on peut les supprimer. 

In [6]:
dfprofiles.dropna(0,subset=['PARTY'],inplace=True)

#### Autres données : 

In [7]:
dfprofiles.MEDIA_PROFESSIONAL.unique()

array([nan,  1.])

On remplace les nan dans `MEDIA_PROFESSIONAL` par 0 
- S'il s'agit d'un media professionel : 1
- S'il s'agit d'un individu : 0

In [8]:
dfprofiles['MEDIA_PROFESSIONAL'].fillna(0,inplace=True)

In [9]:
print('nombre de medias dans le base : ', dfprofiles[dfprofiles['MEDIA_PROFESSIONAL']==1].shape[0])
print("nombre d'individus"," dans le base : ", dfprofiles[dfprofiles['MEDIA_PROFESSIONAL']==0].shape[0])

nombre de medias dans le base :  46
nombre d'individus  dans le base :  19406


**Il y a beaucoup plus d'invidus que de media, ce qui est rassurant compte tenu de l'utilisation que l'on veut faire**

In [10]:
dfprofiles.isna().sum()

FROM_USER_ID             0
PROFILE_NATURE           0
PARTY                    0
MEDIA_PROFESSIONAL       0
SEX                   5338
dtype: int64

Il reste 5338 nans dans les données. 
**On ne s'en occupe pas pour l'instant**

#### Nombre d'utilisateurs uniques  : 

In [11]:
dfprofiles['FROM_USER_ID'].nunique()

19452

# Scraping :  

In [2]:
user_list=[1264925822]
#Moi : 
#user_list=[796492866]

In [3]:
#Boucle for sur tous les ids utilisateurs de la base : 
for u_id in user_list: 
    filename='Donnees/tweets_pol/'+str(u_id)+'.csv'
    #Lancer la configuration : 
    c = twint.Config()
    #On défini l'user : 
    c.User_id=u_id
    #On ne cherche que sur 2017 : 
    
    c.Year=2017
    
    #c.Since='2015-12-20'
    #c.Until="2017-12-28"
    
    #Supprimer cette ligne pour tout récupérer :
    #c.Custom["tweet", "nlikes"] = ["tweet", "nlikes"]
    
    c.Output = filename
    #On stock dans un csv :
    c.Store_csv = True
    #Nous ne souhaitons pas voir les tweets apparaitre dans la console Python
    c.Hide_output = True  
    #Nous ne récupérons pas les hashtags
    c.Show_hashtags = False   
    
    
    
    #c.Store_object = True # nous stockons également les tweets dans un objet
    #On lance la recherche :
    twint.run.Search(c)
    #Tous les tweets seront conservés dans la variable tweets_as_objects
    #tweets_as_objects = twint.output.tweets_object 

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
CRIT

Expecting value: line 1 column 1 (char 0) [x] run.Feed
[!] if get this error but you know for sure that more tweets exist, please open an issue and we will investigate it!


In [4]:
#help(twint.output)

In [5]:
#help(twint.Config)

In [6]:
df=pd.read_csv('Donnees/tweets_pol/'+str(u_id)+'.csv', error_bad_lines=False)

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 500)

In [8]:
df

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,814994099262652417,814990778703319040,1483144709000,2016-12-31,01:38:29,"Paris, Madrid",1264925822,lrrepressible,Guiguilesbonstuyaux,NaN,Et pourquoi ça? Pourquoi ont-ils plus de mérite que les autres?,['msieurleprof'],[],[],0,0,0,[],[],https://twitter.com/lrrepressible/status/814994099262652417,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '1264925822', 'username': 'lrrepressible'}, {'user_id': '379872730', 'username': 'MsieurLeProf'}]",NaN,NaN,NaN,NaN
1,814993901262139392,814991711952928768,1483144662000,2016-12-31,01:37:42,"Paris, Madrid",1264925822,lrrepressible,Guiguilesbonstuyaux,NaN,Vous abusez c'est archi stylé surtout la fin!,['malaisetele'],[],[],0,0,1,[],[],https://twitter.com/lrrepressible/status/814993901262139392,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '1264925822', 'username': 'lrrepressible'}, {'user_id': '4656625635', 'username': 'malaisetele'}]",NaN,NaN,NaN,NaN
2,814638683504734208,813865398022062081,1483059972000,2016-12-30,02:06:12,"Paris, Madrid",1264925822,lrrepressible,Guiguilesbonstuyaux,NaN,https://twitter.com/lrrepressible/status/812003707546112000 …,['pandanadou'],['https://twitter.com/lrrepressible/status/812003707546112000'],[],0,0,0,[],[],https://twitter.com/lrrepressible/status/814638683504734208,False,https://twitter.com/lrrepressible/status/812003707546112000,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '1264925822', 'username': 'lrrepressible'}, {'user_id': '122145388', 'username': 'PandaNadou'}]",NaN,NaN,NaN,NaN
3,814637636593848320,814438822582845440,1483059722000,2016-12-30,02:02:02,"Paris, Madrid",1264925822,lrrepressible,Guiguilesbonstuyaux,NaN,Ça a l'air archi drôle mais quelqu'un m'explique?,['gloryowlcomix'],[],[],0,0,0,[],[],https://twitter.com/lrrepressible/status/814637636593848320,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '1264925822', 'username': 'lrrepressible'}, {'user_id': '1649962250', 'username': 'GloryOwlComix'}]",NaN,NaN,NaN,NaN
4,812013763129917440,812003707546112000,1482434142000,2016-12-22,20:15:42,"Paris, Madrid",1264925822,lrrepressible,Guiguilesbonstuyaux,NaN,https://twitter.com/lrrepressible/status/812001754808250368 …,['lrrepressible'],['https://twitter.com/lrrepressible/status/812001754808250368'],[],0,0,0,[],[],https://twitter.com/lrrepressible/status/812013763129917440,False,https://twitter.com/lrrepressible/status/812001754808250368,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '1264925822', 'username': 'lrrepressible'}]",NaN,NaN,NaN,NaN
5,812003934466441216,811994668691058688,1482431798000,2016-12-22,19:36:38,"Paris, Madrid",1264925822,lrrepressible,Guiguilesbonstuyaux,NaN,https://twitter.com/lrrepressible/status/812003707546112000 …,['tony_st_laurent'],['https://twitter.com/lrrepressible/status/812003707546112000'],[],0,1,1,[],[],https://twitter.com/lrrepressible/status/812003934466441216,False,https://twitter.com/lrrepressible/status/812003707546112000,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '1264925822', 'username': 'lrrepressible'}, {'user_id': '275124718', 'username': 'tony_st_laurent'}]",NaN,NaN,NaN,NaN
6,812003928367923200,811958435508088832,1482431797000,2016-12-22,19:36:37,"Paris, Madrid",1264925822,lrrepressible,Guiguilesbonstuyaux,NaN,https://twitter.com/lrrepressible/status/812003707546112000 …,[],['https://twitter.com/lrrepressible/status/812003707546112000'],[],0,0,0,[],[],https://twitter.com/lrrepressible/status/812003928367923200,False,https://twitter.com/lrrepressible/status/812003707546112000,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '1264925822', 'username': 'lrrepressible'}]",NaN,NaN,NaN,NaN
7,812003920767844358,811983087911534592,1482431795000,2016-12-22,19:36:35,"Paris, Madrid",1264925822,lrrepressible,Guiguilesbonstuyaux,NaN,https://twitter.c

### code d'exemple : 

In [ ]:
for compte in compte_twitter:
 #nom du fichier csv qui sera produit
     filename = compte+".csv"
 #lancement de l’objet twint
     c = twint.Config()
 #nom du compte que nous recherchons
     c.Username = compte
 #Nous désirons stocker un fichier csv contenant l’ensemble des tweets récupérés
     c.Store_csv = True
     # Nombre de tweets récupérés
     c.Count = True
 #Si nous avons déjà un fichier csv des tweets, nous n’analysons pas le compte Tweeter de la personnalité
     if os.path.exists(filename):
         print('déjà présent :' + compte )
 #Sinon, nous récuperons la liste des tweets
     else:
         c.Custom["tweet"] = ["tweet"]
         c.Output = filename
 #Nous ne souhaitons pas voir les tweets apparaitre dans la console Python
         c.Hide_output = True  
 #Nous ne récupérons pas les hashtags
         c.Show_hashtags = False     
         c.Store_object = True # nous stockons également les tweets dans un objet  
 #Lancement de la recherche
         twint.run.Search(c)
 #Tous les tweets seront conservés dans la variable tweets_as_objects
         tweets_as_objects = twint.output.tweets_object 

Erreur :

In [ ]:
#Same error and just following the simple code of query search and save it in a panda.

import twint
import pandas as pd
c = twint.Config()
c.Search = "agroecología"
c.Pandas = True
twint.run.Search(c)
Tweets_df = twint.storage.panda.Tweets_df
Tweets_df.to_csv(r'Query2ES.csv', index = None, header=True)
twint.storage.panda.clean

## From : https://towardsdatascience.com/analyzing-tweets-with-nlp-in-minutes-with-spark-optimus-and-twint-a0c96084995f

In [ ]:
c=twint.Config()
#c.Search = "data"
c.User_id=1264925822
# Custom output format
#c.Format = "Username: {username} |  Tweet: {tweet}"
c.Limit = 1
c.Year=2017
c.Pandas = True

twint.run.Search(c)
Tweets_df = twint.storage.panda.Tweets_df

In [ ]:
Tweets_df.head()

In [ ]:
def available_columns():
    return twint.output.panda.Tweets_df.columns

In [ ]:
available_columns()

In [ ]:
username='adrienhans'

In [1]:
import twint

c = twint.Config()
c.Limit = 20
c.Username = username
c.Pandas = True
twint.run.Search(c)
Tweets_df = twint.storage.panda.Tweets_df

NameError: name 'username' is not defined

In [ ]:
help(twint.run.Search)

# Other data : 

In [2]:
import pandas as pd

In [3]:
df_fi_tweets_id=pd.read_csv('Donnees/data_elysee2017_tweets/posts_ids_fi.tab', sep='\t')

In [5]:
df_fi_tweets_id.shape

(2192544, 1)

In [5]:
import twint

In [10]:
#c.User_full = True
help(twint.Config().User_id)

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.

